In [28]:
import sys
sys.path.append('/workspaces/CLS-Augur')
sys.path.append('../Augur')
from Augur.rag import GraphRAG, SparQLRAG
import pandas as pd

ontology_files = [
    '../datafiles/dbpedia_2016-10.owl'
]

model = "sentence-transformers/all-mpnet-base-v2"

df_train = pd.read_json('../datafiles/train-data.json')

ontology_db = GraphRAG(model, ontology_files)

                       
                        
query = "This property describes the relationship between a musical work and the populated place where it was recorded. It has domain 'MusicalWork' and range 'PopulatedPlace'. The property is labeled in multiple languages including English, Dutch, Greek, and French. It is a subproperty of 'coparticipatesWith' from the DUL ontology."
output = ontology_db.process_query(query, 5)


In [29]:
from rdflib import Graph

dbpedia_graph = Graph().parse('../datafiles/dbpedia_2016-10.owl', format='xml')

dbpedia_ids = {element for triple in dbpedia_graph for element in triple}

In [30]:
graph  = Graph().parse(data=output, format='turtle')

In [31]:
gold_standard = pd.read_csv('../datafiles/lcquad_test_gold.csv')

In [32]:
gold_standard.sparql_query[0]

' SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/Marine_Corps_Air_Station_Kaneohe_Bay> <http://dbpedia.org/property/architect> ?uri. <http://dbpedia.org/resource/New_Sanno_Hotel> <http://dbpedia.org/ontology/tenant> ?uri} '

In [33]:
import re

pattern = r'<(http[s]?://[^ >]*dbpedia\.org[^ >]*)>'

In [34]:
query = set(re.findall(pattern, gold_standard.sparql_query[0]))

In [35]:
print(query)

{'http://dbpedia.org/resource/New_Sanno_Hotel', 'http://dbpedia.org/ontology/tenant', 'http://dbpedia.org/resource/Marine_Corps_Air_Station_Kaneohe_Bay', 'http://dbpedia.org/property/architect'}


In [48]:
for s,p,o in dbpedia_graph:
    if 'resource' in p:
        print(o)

In [26]:
gold_standard.output[0]

"{'head': {'link': [], 'vars': ['uri']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/United_States_Navy'}}]}}"

In [51]:
gold_standard.sparql_query[1]

'SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/international> <http://dbpedia.org/resource/Muslim_Brotherhood> . ?x <http://dbpedia.org/ontology/religion> ?uri  . ?x <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/PoliticalParty>}'